# Введення в реляційні бази даних.
# PostgreSQL + Python

---

## лекція 4: Інші об'єкти бази даних: представлення, функції.

In [ ]:
from python_postgresql import execute_query, execute_read_query, connection
print("лекція 4: Інші об'єкти бази даних: представлення, функції.")

### Представлення - VIEW.

До цього єдиним об'єктом БД з яким ми працювали були таблиці. Але БД - досить складні комплекси з великою кількістю завдань і можливостей, щоб працювати з одним типом об'єктів.

Представлення - VIEW - це збережений запит формі об'єкта БД (така собі віртуальна таблиця).
Так як це таблиця, то ми можемо робити до неї SELECT, JOIN - і використовувати в будь-яких операціях для отримання інформації.
Швидкість виконання цих запитів буде такою ж, як і з реальною таблицею (+\-).

Для чого це можна використовувати:
- існують методи кешування частих запитів користувачів (використовуються на великих БД із значними обсягами даних і потоком запитів) які називають "матеріалізація". Це допомагає суттєво підвищити швидкість обробки запитів.
-  дає можливість спростити складні запити - коли часто використовуємий підзапит реалізується як VIEW і ви просто ніби працюєте з існуючою таблицею (хоча, по суті, це підзапит)
- допомагає спростити роботу якогось прошарку користувачів, для яких є характерним якийсь конкретний запит: ви створюєте для них VIEW на основі запиту будь-якої складності. І далі вони ніби працюють з однією таблицею в якій є все, що їм необхідно, отримуючи інформацію за допомогою простих запитів. Це може бути актуально коли користувач використовує ORM і для спрощення використання зручно підготувати для нього структури даних (хай і віртуальні), з якими просто працювати.
- дозволяє закривати частину колонок від користувачів - створивши для таблиць VIEW які надають їм тільки ту інформацію, до якої у них повинен бути доступ. Можливо обмежити за допомогою фільтрів доступ до якихось рядків в таблицях.



Представлення (VIEW) бувають:
- тимчасові TEMPORARY або TEMP
    - з такою вказівкою уявлення створюється як тимчасове. Тимчасові вистави автоматично видаляються наприкінці сеансу. Існуюче постійне уявлення з тим самим ім'ям не буде видно в поточному сеансі, доки існує тимчасове, проте до нього можна звернутися, доповнивши ім'я вказівкою схеми. Якщо у визначенні уявлення задіяні тимчасові таблиці, уявлення створюється як тимчасове (незалежно від присутності явної вказівки TEMPORARY).
- рекурсивні - RECURSIVE
    - використовують для створення VIEW на основі рекурсивних запитів. Ми не будемо в нашому курсі заглиблюватись в це питання.
- оновлювані. Це, умовно кажучі, самі "звичайні" представлення з якими ми будемо в основному працювати.
- матеріалізуємі представлення - матеріалізовані уявлення в Postgres Pro засновані на системі правил, як і звичайні уявлення, але їх вміст зберігається як таблиця. Це можна уявити як представлення, результати запитів до якого зберігаються як таблиця ("матеріалізуються") - ніби кешуються, для прискореного доступу для частих запитів. Зручно для часто виконуємих запитів для не дуже часто змінюємих даних, для прискорення роботи з віддаленими БД. Ми не будемо в рамках нашого курсу вивчати такі типи представлень.


#### Створення представлень

[Докладно тут](https://www.postgresql.org/docs/current/sql-createview.html)

Так виглядає визначення VIEW в офіційній документації:

```SQL
CREATE [ OR REPLACE ] [ TEMP | TEMPORARY ] [ RECURSIVE ] VIEW name [ ( column_name [, ...] ) ]
    [ WITH ( view_option_name [= view_option_value] [, ... ] ) ]
    AS query
    [ WITH [ CASCADED | LOCAL ] CHECK OPTION ]
```

Почнемо покроково - із більш спрощених варіантів:
1) просте уявлення, створене з однієї таблиці, яке повертає нам лише два атрибута з цієї таблиці:

```SQL
CREATE VIEW view_name AS 
SELECT column1, column2 
FROM table_name;
```

2) уявлення, створене з шляхом поєднання двохм таблиць:

```SQL
CREATE VIEW view_name AS
SELECT column1, column2, column3 
FROM table1 
JOIN table2 
ON table1.id = table2.id;
```

2) ну і третє представлення, в якому для формування одного з стовпчиків фінального набору використовується підзапит:

```SQL
CREATE VIEW view_name AS 
SELECT column1, column2, 
       (SELECT SUM(column3) 
        FROM table2 
        WHERE table1.id = table2.id) AS sum_column3 
FROM table1;
```

А тепер спробуємо ці запити реалізувати на нашій БД(перевірте - щоб всі контейнери були запущені):

In [ ]:
operation_query = """
CREATE VIEW customer_orders AS 
SELECT orders.order_id, orders.customer_id, orders.order_date 
FROM orders;
"""

final_set = execute_query(connection, operation_query)

Звертаю Вашу вагу - що цей запит створює об'єкт БД і не повертає результатів запиту - тому використовуємо функцію execute_query().
Це представлення повертає нам __id__ замовлення, ідентифікатор користувача і дату створення замовлення.
Тепер ми можемо стоврювати запити до створеної нами VIEWS як до звичайної таблиці:

In [ ]:
operation_query = """
SELECT * 
FROM customer_orders;
"""

final_set = execute_read_query(connection, operation_query)
if final_set:
    for record in final_set:
        print(record)
else:
    print(final_set)

для другого приклада:

In [ ]:
operation_query = """
CREATE VIEW customer_order_details AS 
SELECT orders.order_id, orders.customer_id, order_details.product_id, order_details.quantity 
FROM orders 
JOIN order_details 
ON orders.order_id = order_details.order_id;
"""

final_set = execute_query(connection, operation_query)

і тепер робимо запит до створеного об'єкта БД:

In [ ]:
operation_query = """
SELECT * 
FROM customer_order_details 
WHERE customer_id = 'ALFKI';
"""

final_set = execute_read_query(connection, operation_query)
if final_set:
    for record in final_set:
        print(record)
else:
    print(final_set)

ну і для останнього прикладу:

In [ ]:
operation_query = """
CREATE VIEW customer_order_total AS 
SELECT orders.order_id, orders.customer_id, 
       (SELECT SUM(order_details.quantity * order_details.unit_price) 
        FROM order_details 
        WHERE orders.order_id = order_details.order_id) AS order_total 
FROM orders;
"""

final_set = execute_query(connection, operation_query)

і дивимось на результат запиту до цьго представлення:

In [ ]:
operation_query = """
SELECT customer_id, SUM(order_total) AS total 
FROM customer_order_total 
GROUP BY customer_id;
"""

final_set = execute_read_query(connection, operation_query)
if final_set:
    for record in final_set:
        print(record)
else:
    print(final_set)

Давайте розберемо деякі опції створення VIEW.
Якщо ви подивитесь на визначення в документації (на початку розділа), то ви побачите можливість опціонального викоритання наступних конструкцій:
- ```[ OR REPLACE ]``` - можливо, в попередніх прикладах ви намагались запустити неодноразово блоки коду, де ми створювали в нашій БД VIEW. Якщо так - то ви повинні були отрати помилку (окрім першого запуску) з повідомленням ```DuplicateTable: relation "customer_orders" already exists```
Це сталось тому що в перший раз представлення було створене, а потім ми намались створити повторно представлення з тим же самим ім'ям. СУБД бачить що об'єкт з таким ім'ям вже існує і повертає помилку. Якщо Вам необхідно перевизначити це представлення - використовуйте ON REPLACE. 
Команда ```CREATE OR REPLACE VIEW``` діє так само, але якщо подання з цим ім'ям вже існує, воно замінюється. Новий запит повинен видавати ті ж стовпці, що видавав запит, раніше визначений для цього подання (тобто стовпці з такими ж іменами повинні мати ті ж типи даних і слідувати в тому ж порядку), але може додати кілька нових стовпців в кінці списку. Обчислення, у яких формуються стовпці уявлення, може бути зовсім іншими.

Наш з Вами перший приклад можна перевизначити так (я додав ще один атрибут - але це не обов'язково):

In [ ]:
operation_query = """
CREATE OR REPLACE VIEW customer_orders AS 
SELECT orders.order_id, orders.customer_id, orders.order_date, orders.ship_address 
FROM orders;
"""

final_set = execute_query(connection, operation_query)

In [ ]:
operation_query = """
SELECT * 
FROM customer_orders;
"""

final_set = execute_read_query(connection, operation_query)
if final_set:
    for record in final_set:
        print(record)
else:
    print(final_set)

Зверніть увагу, що представлення накладають деякі обмеження, наприклад стосовно зміни:
- для існуючого представлення можливо лише додавати нові стовпці
    - неможливо видаляти існуючи стовпці
    - неможливо змінювати імена стовпців
    - неможливо замінити порядок слідування стовпців

Але - ви можете змінити ім'я VIEW за допомогою синтаксису:
```SQL
ALTER VIEW  old_view_name RENAME TO new_view_name
```

Ну і нічого не заважає нам видаляти існуючи VIEW:

```SQL
DROP VIEW [IF EXIST] view_name
```

Так як VIEW - це об'єкт бази даних - Ви можете їх побачити. Використовуючи свій клієнт підключений до БД (pgAdmin або DBeaver в нашому прикладі) в розділі "Schemas.public.Views".

- TEMPORARY або TEMP

З такою вказівкою уявлення створюється як тимчасове. Тимчасові представлення (VIEW) автоматично видаляються наприкінці сеансу. Існуюче постійне уявлення з тим самим ім'ям не буде видно в поточному сеансі, доки існує тимчасове, проте до нього можна звернутися, доповнивши ім'я вказівкою схеми.

Якщо у визначенні уявлення задіяні тимчасові таблиці, уявлення як і створюється як тимчасове (незалежно від присутності явного вказівки TEMPORARY).

##### Перейменування VIEW:

```SQL

ALTER VIEW old_name RENAME TO new_name

```

##### Видалення VIEW:

```SQL

DROP VIEW [IF EXIST] view_name

``

##### [Модифікація даних через VIEW](https://www.postgresql.org/docs/15/sql-createview.html)

Прості уявлення стають змінними автоматично: система дозволить виконувати команди INSERT, UPDATE та DELETE з таким поданням так само, як і зі звичайною таблицею. Подання буде автоматично змінюватися, якщо воно задовольняють одночасно всім наступним умовам:

    - Список FROM у запиті, що визначає поданні, повинен містити рівно один елемент, і це повинна бути таблиця або інше подання, що змінюється.

    - Визначення подання не повинно містити пропозиції WITH, DISTINCT, GROUP BY, HAVING, LIMIT та OFFSET на верхньому рівні запиту.

    - Визначення подання не повинно містити операції з безліччю (UNION, INTERSECT та EXCEPT) на верхньому рівні запиту.
    
    - Список вибірки у запиті не повинен містити агрегатні та віконні функції, а також функції, що повертають безліч.

Автоматично оновлюване уявлення може містити як стовпці, що змінюються, так і не змінюються. Стовпець буде змінним, якщо це просте посилання змінюваний стовпець нижчележачого базового відносини; в іншому випадку цей стовпець буде доступний тільки для читання, і якщо команда INSERT або UPDATE спробує записати значення в нього, виникне помилка.

Якщо представлення включає умову WHERE - то ця умова обмежує рядки, які можуть бути змінені командою UPDATE. Більше того - така команда може змінити рядок так, що він більше не буде доступний через це представлення. 
Ви можете побачити логіку в ціх обмеженнях - якщо єлемент (запис), який ви намагаєтесь модифікувати\створити\видалити в представленні може бути однозначно співставлений з базовими таблицями - то це зробити можливо.

Більш складні представлення, які не задовольняють цим умовам, за замовченням доступні лише читання: система не дозволить виконати операції створення, зміни чи видалення рядків у такому представленні. Створити ефект зміненого уявлення для них можна, визначивши тригери INSTEAD OF, які перетворюватимуть запити на зміну даних у відповідні дії з іншими таблицями. Про тригери ми будемо розмовляти пізніше.

Є досить багато можливостей і нюансів використання уявлень - прошу Вас за ними звернуся, при необхідності, в документацію

### Функції в PostgreSQL

Функції - це також об'єкти БД, які можуть приймати аргументи і повертати результати.

Зазвичай все, що ми можемо описати і зробити за допомогою функцій БД, ви можете зробити і з зовнішнього застосунку - наприклад з вашого застосунку який написаний на Python: отримаєте необхідні первинні дані і зробите з ними те, що Вам необхідно. Але у багатьох випадках створення функції на стороні БД буде кращім вибором. Причинами для цього може бути наступне:
- функції (або зберігаємі процедури) компіюються і зберігаються на стороні БД, тобто їх виклик і виконання - може бути значно швидшим ("дешевшим") ніж зовнішня реалізація того ж функціонала
- згідно з принципом [SRP (single responsibility principle)](https://uk.wikipedia.org/wiki/%D0%9F%D1%80%D0%B8%D0%BD%D1%86%D0%B8%D0%BF_%D1%94%D0%B4%D0%B8%D0%BD%D0%BE%D1%97_%D0%B2%D1%96%D0%B4%D0%BF%D0%BE%D0%B2%D1%96%D0%B4%D0%B0%D0%BB%D1%8C%D0%BD%D0%BE%D1%81%D1%82%D1%96) - те, що обробляє кортежі даних логічно зберігати ближче до даних
- якщо різні клієнтські застосунки потребують одних і тих же дій з даними - логічно не писати в кожному із застосунків цю обробку, а написати її на рівні БД - і хай всі користуються
- керування безпекою даних - ви можете за допомогою можливостей БД по керуванню доступом до об'єктів БД обмежити доступ до функцій тім користувачам\групам і т.ін, коли вважаєте за необхідне
- знеження кількості запитів і трафіку в мережі - через реалізацію первинної обробки на стороні сервера БД
- розділення розробки на зони відповідальності: точно так, як розділяють frontend\backend розробку, можна визначити розробку на стороні сервера БД - і при наявності грамотного архітектора і фахівця з БД ви отримаєте більш ефективний і стабільний результат - завдяки більш глибокій експертизі виконавців в своїх зонах відповідальності
- модульність створення коду: якщо Вам у декількох ваших SQL-запитах потрібна одна і та ж дія з даними - розумно стоврити такий код один раз і перевикористовувати його


Ви вже зустрічали і використовували вбудовані функції PostgreSQL. Наприклад:
- __AVG__ і __SUM__ - повертає середнє значення і суму для набора даних відповідно

```SQL
SELECT AVG(column_name) 
FROM table_name;

SELECT SUM(column_name) 
FROM table_name;
```

- __COUNT__ - повертає кількість рядків

```SQL
SELECT COUNT(*) 
FROM table_name;
```

- __MAX__ і __MIN__ - повертає, відповідно, максимальне і мінімальне значення з набору даних

```SQL
SELECT MAX(column_name) 
FROM table_name;

SELECT MIN(column_name) 
FROM table_name;
```

- __UPPER__ і __LOVER__ - перетворює, відповідно, символи в рядках в uppercase і lowercase

```SQL
SELECT UPPER(column_name) 
FROM table_name;

SELECT LOWER(column_name) 
FROM table_name;
```

- __SUBSTRING__  - повертає субрядок

```SQL
SELECT SUBSTRING(column_name, start, length) 
FROM table_name;
```

- __TRIM__  - видаляє пробіли на початку і після рядка

```SQL
SELECT TRIM(column_name) 
FROM table_name;
```

- __NOW__  - повертає поточні час і дату

```SQL
SELECT NOW();
```

Це далеко не повний перелік функцій і більш докладно ви можете [познайомитись з ними в документації.](https://postgrespro.ru/docs/postgresql/15/xfunc-internal)
Але - вбудовані функції - це далеко не все. Ви, також, можете створювати самостійно свої функції в PostgreSQL.
Для цього PostgreSQL надає Вам декілька мов програмування і надає досить зрозумілі можливості для підключення інших процедурних мов програмування.

[У PostgreSQL ви можете писати власні функції за допомогою кількох мов, зокрема:](https://postgrespro.ru/docs/postgresql/15/xfunc)
- [SQL: декларативна мова, яка використовується для керування та обробки даних у реляційних базах даних.](https://www.postgresql.org/docs/current/xfunc-sql.html#XFUNC-SQL-BASE-FUNCTIONS)
- [PL/pgSQL: процедурна мова, спеціально розроблена для використання з PostgreSQL.](https://www.postgresql.org/docs/current/plpgsql.html)
- [PL/Python: процедурна мова, яка використовує мову програмування Python.](https://www.postgresql.org/docs/current/plpython.html)
- [PL/Tcl: процедурна мова, яка використовує мову сценаріїв Tcl.](https://www.postgresql.org/docs/current/pltcl.html)
- [PL/Perl: процедурна мова, яка використовує мову програмування Perl.](https://www.postgresql.org/docs/current/plperl.html)
- [C: мова програмування низького рівня.](https://www.postgresql.org/docs/current/xfunc-c.html)

Ми не будемо вивчати всі можливості створення і використання функцій в рамках нашого курсу. Ми познайомимось з SQL-функціями і зовсім небагато поговоримо про PL/pgSQL.

Будь-яка функція в Postgres:
- складається з набора тверджень і повертає результат останнього
- можуть виконувати SELECT, INSERT, UPDATE, DELETE (CRUD-операції)
- не можуть включати COMMIT, SAVEPOINT (TCL), VACUUM (utility)

Стосовно останнього - в рамках нашого короткого курсу ми не вивчаємо лише базові теми і залишаємо поза обговоренням досить багато тем, які було б добре знати і розуміти) Одна з таких тем - транзакції. [Що це - читайте тут.](https://uk.wikipedia.org/wiki/%D0%A2%D1%80%D0%B0%D0%BD%D0%B7%D0%B0%D0%BA%D1%86%D1%96%D1%8F_(%D0%B1%D0%B0%D0%B7%D0%B8_%D0%B4%D0%B0%D0%BD%D0%B8%D1%85))

Розумійте транзакції як блок операцій, який повинен бути виконаний лише в повному обсязі і коли він виконується то може блокувати якісь операції над деякими записами\атрибутами і т.ін. Це робиться щоб, наприклад, уникнути конкуренції між різними процесами по доступу до даних, забезпечити цілсність даних при можливих збоях і т. ін. Тобто - якщо блок операцій, який може включати в собі декілька операцій, не зробила успішно їх всі - то вся операція (транзакція) з усіма окремими лпераціями в її складі - не відбувається.

Так от остання твердження стосовно функцій - про транзакції в тому разі. Треба розуміти - що функції самі по собі - авторанзакційні. Тобто - якщо функція почала виконання і чомусь не завершила його коректно - то зміни, які були внесені - "відкатяться".


#### Синтаксис визначення функції

[Докладніше про це можна прочитати тут](https://www.postgresql.org/docs/15/sql-createfunction.html), ми ж будемо використовувати більш спрощені і обмежені варіанти використання.

```SQL
CREATE FUNCTION name_of_func([arg_1, arg_2, ..., arg_N]) RETURNS type_of_data 
AS $$
-- logic
$$ LANGUAGE name_of_lang
```

Знак ```$$``` може бути замінений на лапки, але такий варіант здається мені більш зручним (лапки потрібно в тілі функції екранувати, що знижує читабельність).
Можна використовувати ```CREATE OR REPLACE name_of_func ...``` - це буде діяти аналогічно подібній конструкції в представленнях і мати аналогічні обмеження.

##### Функція без параметрів

Можливо - це буде не найкорисніша функція, але вона повинна продемонструвати ідею:

```SQL
CREATE OR REPLACE FUNCTION order_count()
RETURNS INTEGER AS $$
    SELECT COUNT(*) FROM Orders;
$$ LANGUAGE SQL;
```

і приклад виклику цієї функції:

```SQL
SELECT order_count();
```

А тепер давайте вирішим більш реальне завдання: в нашій таблиці customers є атрибут customers.fax. Він не завжди має значення - тобто чатина рядків для цього атрибута має значення Null. Давайте напишемо функцію яка буде замінювати значення Null на запис "do not use fax". Пропроую не чіпати основну БД, а створити копію таблиці customers яку назвати tmp_customers:

```SQL
SELECT *
INTO tmp_customers
FROM customers;
```

Звісно ми можемо це зробитиза допомогою звичайного оператора UPDATE, щось на кшталт:
```SQL
UPDATE tmp_customers
SET fax = "do not use fax"
WHERE fax is Null
```
але давайте оформимо все це як функцію (можливо ми повинні надати для користувача БД саме функцію, а не можливість виконувати будь-які запити на таблицях).

Створимо копію таблиці customers:

In [ ]:
operation_query = """
SELECT *
INTO tmp_customers
FROM customers;
"""

final_set = execute_query(connection, operation_query)

Перевірте в БД наявність нової таблиці будь-яким зручним для вас методом і зверніть увагу на наявність в деяких рядках значення Null для атрибуту tmp_customers.fax

Тепер - створимо функцію:

In [4]:
operation_query = """
CREATE OR REPLACE FUNCTION update_customer_fax() RETURNS void AS $$
    UPDATE tmp_customers
    SET fax = 'do not use fax'
    WHERE fax is Null
$$ LANGUAGE SQL;
"""

final_set = execute_query(connection, operation_query)

Query executed successfully


Тепер перейдіть в обраний Вами клієнт (pgAdmin або DBeaver), оновіть стан БД і подивіться - що у розділі "Функції" у Вас з'явився новий об'єкт. Зверніть увагу і на Вашу таблицю tmp_customers - вона не точно така як була до цього.
Ну а тепер можна просто викликати функцію для конання:

In [5]:
operation_query = """
SELECT update_customer_fax();
"""

final_set = execute_query(connection, operation_query)

Query executed successfully


Тепер подивіться на модифіковану таблицю - зміни внесено.

##### Функції з аргументами.


Початок визначення функції (з документації):
```SQL
CREATE [ OR REPLACE ] FUNCTION
    name ( [ [ argmode ] [ argname ] argtype [ { DEFAULT | = } default_expr ] [, ...] ] )
    [ RETURNS rettype
      | RETURNS TABLE ( column_name column_type [, ...] ) ]
...
...
...
```

Розширимо знання про функції. Давайте розберемо можливість передачі аргументів. Аргументи бувають:
argmode:
- IN - вхідні аргументи. Це значення за замовченням. Тобто - якщо нічого не вказано - значить режим цього аргумента - вхідний
- OUT - вихідний аргумент
- INOUT - аргумент і вхідний і вихідний
- VARIADIC - масив вхідних аргументів (ми не будемо розбирате в рамках курсу)
argname:
- необов'язкове ім'я для аргумента, яке може використовуватись в функції
argtype:
- обов'язковий параметр - тип аргумента
DEFAULT - необов'язковий параметр, який описує значення аргумента якщо воно не було передано в функцію.
rettype:
- тип результату
column_name column_type:
- відповідно ім'я колонки і її тип - коли ви повертаєте не скалярний результат, а цілу таблицю

Є деякі правила використання та визначення аргументів функцій в PostgreSQL:

- Аргументи мають бути визначені явно: у PostgreSQL усі аргументи функції мають бути визначені за допомогою типу даних. Це дозволяє базі даних перевіряти типи аргументів під час виклику функції.

- Можна вказати значення за замовчуванням: якщо для аргументу вказано значення за замовчуванням, функція використовуватиме це значення, якщо аргумент не буде надано під час виклику функції.

- Аргументи можуть мати різні режими: PostgreSQL підтримує різні режими для аргументів функції, зокрема IN, OUT та INOUT. Аргументи IN використовуються для передачі значень у функцію, аргументи OUT використовуються для повернення значень із функції, а аргументи INOUT використовуються для передачі значень у функцію та повернення значень із них.

- Назви аргументів є необов’язковими: хоча найкращою практикою є надання описових імен для аргументів функції, назви аргументів є необов’язковими в PostgreSQL.

- Перезавантаження функцій не підтримуються: на відміну від деяких інших систем баз даних, PostgreSQL не підтримує перевизначення існуючих функцій в будб-який спосіб. Це означає, що функції з однаковою назвою не можуть мати різні списки аргументів.

Аргументи можна передавати за значенням або за посиланням: у PostgreSQL аргументи можна передавати за значенням або за посиланням. Коли аргумент передається за значенням, копія аргументу передається функції, тоді як коли аргумент передається за посиланням, вказівник на аргумент передається функції.

Давайте перейдемо до практики і модифікуємо написну нами раніше функцію, яка замінювала для атрибута tmp.customers.fax значення Null на "do not use fax". Давайте перетворимо її на функцію, яка отримує значення яке треба замінити і значення на яке заміняти. Тобто - використовуючи цю функцію користувач зможе самостійно вирішувати який варіант значення має бути для, наприклад, випадку відсутності факсу.


In [2]:
operation_query = """
CREATE OR REPLACE FUNCTION update_customer_fax(old_value varchar(24), new_value varchar(24)) RETURNS void AS $$
    UPDATE tmp_customers
    SET fax = new_value
    WHERE fax = old_value
$$ LANGUAGE SQL;
"""

final_set = execute_query(connection, operation_query)

Query executed successfully


Тепер виконаємо створену функцію:

In [6]:
operation_query = """
SELECT update_customer_fax('do not use fax', 'no fax')
"""

final_set = execute_query(connection, operation_query)

Query executed successfully


Ну і давайте подивимось на результат:

In [ ]:
operation_query = """
SELECT fax
FROM tmp_customers;
"""

final_set = execute_read_query(connection, operation_query)
if final_set:
    for record in final_set:
        print(record)
else:
    print(final_set)

Тепер давайте розберемо функцію з більш складною структурою. Наприклад - я ми б хотіли створити функцію яка буде отримувати на вхід рік і місяць і повертати рейтинг співробітників компанії по продажах за цей період.
SQL запит для цього міг би мати наступний вигляд:
```SQL
SELECT 
  employees.employee_id, 
  employees.first_name, 
  employees.last_name , 
  SUM(customer_order_total.order_total) AS total_sales
FROM 
  employees
  JOIN orders ON employees.employee_id = orders.employee_id
  join customer_order_total on customer_order_total.order_id = orders.order_id  
WHERE 
  orders.order_date  BETWEEN '1997-01-01' AND '1997-07-01'
GROUP BY 
  employees.employee_id, 
  employees.first_name, 
  employees.last_name
 order by total_sales desc;
```
Тут в аргументах BETWEEN необхідно підставляти відповідні дати (прошу задуматись - а чи вірно побудовани умоіи запиту? Як BETWEEN враховує границі?)

Але ми ускладнимо завдання - функція повинна отримувати в якості вхідних параметрів рік і номер місяця, а повертати відсортовану по продажах таблицю, яка включає дані по співробітнику - id, first_name, last_name, total_sales.

Я пропоную Вам самостійно розібратись з функцією (описана нижче) і на наступній лекції сформулювати питання стосовно незрозумілих моментів.
Для полегшення цієї роботи:
- [створення функцій в документації описано тут](https://www.postgresql.org/docs/15/sql-createfunction.html)
- [типи даних для опису часу описані тут (вам потрібно розібратись з типами data та interval)](https://www.postgresql.org/docs/15/datatype-datetime.html)
- [використані функції для роботи з data та interva описані тут (використана функція make_date())](https://www.postgresql.org/docs/15/functions-datetime.html)

In [7]:
operation_query = """
CREATE OR REPLACE FUNCTION SalesResultsByMonth (
  year_in INT, 
  month_in INT
)
RETURNS TABLE (employee_id int2, first_name varchar(10), last_name varchar(20), total_sales real)
AS $$
SELECT 
  employees.employee_id, 
  employees.first_name, 
  employees.last_name, 
	SUM(customer_order_total.order_total) AS total_sales
FROM 
  employees
  JOIN orders ON employees.employee_id = orders.employee_id
  join customer_order_total on customer_order_total.order_id = orders.order_id  
WHERE 
  make_date(year_in, month_in, 1) <= orders.order_date and orders.order_date < make_date(year_in, month_in, 1) + interval '1 month'   
GROUP BY 
  employees.employee_id, 
  employees.first_name, 
  employees.last_name
 order by total_sales desc
$$ LANGUAGE SQL;
"""

final_set = execute_query(connection, operation_query)

Query executed successfully


Пропоную використати створену нами функцію. Давайте тільки спочатку з'ясуємо інтервал часу який описує наша БД - тобто коли рахунки виставлялись в має сенс оцінювати продавців.
Зробимо простий запит:

In [8]:
operation_query = """
select min(order_date), max(order_date)
from orders;
"""

final_set = execute_read_query(connection, operation_query)
if final_set:
    for record in final_set:
        print(record)
else:
    print(final_set)

(datetime.date(1996, 7, 4), datetime.date(1998, 5, 6))


Тепер, коли ми розуміємо інтервал в якому доцільно оцінювати роботу менеджерів, давайте це зробимо:

In [9]:
operation_query = """
select SalesResultsByMonth(1996, 9);
"""

final_set = execute_read_query(connection, operation_query)
if final_set:
    for record in final_set:
        print(record)
else:
    print(final_set)

('(1,Nancy,Davolio,6883.7)',)
('(8,Laura,Callahan,5248)',)
('(6,Michael,Suyama,4465.6)',)
('(4,Margaret,Peacock,3575.1)',)
('(2,Andrew,Fuller,2950.8)',)
('(3,Janet,Leverling,1762)',)
('(5,Steven,Buchanan,1420)',)
('(7,Robert,King,1330.7999)',)


Зверніть увагу на формат відповіді і цьому випадку - ми отримали record.
Це не завжди зручно. Якщо звернутсь до яункції як до таблиці (функція є повертає нам таблицю), то результат буде виглядати більш звично:

In [10]:
operation_query = """
select *
from SalesResultsByMonth(1997, 5);
"""

final_set = execute_read_query(connection, operation_query)
if final_set:
    for record in final_set:
        print(record)
else:
    print(final_set)

(3, 'Janet', 'Leverling', 18639.8)
(1, 'Nancy', 'Davolio', 9168.25)
(4, 'Margaret', 'Peacock', 7573.2)
(7, 'Robert', 'King', 6041.25)
(5, 'Steven', 'Buchanan', 5127.5)
(8, 'Laura', 'Callahan', 4792.6)
(2, 'Andrew', 'Fuller', 4589.6)
(6, 'Michael', 'Suyama', 751.7)
(9, 'Anne', 'Dodsworth', 139.8)


#### Процедурні мови

До цього ми використовували чистий SQL для створення функцій\зберігаємих процедур. Це може бути не завжди досить функціонально, тому, як ми вже говоримли раніше, PostgreSQL надає ще декілька процедурних мов.

Існує багато думок стосовно того - яка мова буде більш ефективна. Розповсюджена думка - PL/pgSQL є процедурною мовою за замовчуванням у PostgreSQL і тісно інтегрована з системою баз даних. Він вважається швидким, ефективним і безпечним, що робить його хорошим вибором для багатьох програм. До того ж - на продуктивність функцій PL/Python, PL/Tcl, PL/Perl можуть вплинути накладні витрати на серіалізацію та десеріалізацію даних між базою даних та інтерпретатором Python/Tcl/Perl.

В той же час існує значна кількість статей, де автори порінюють швидкість виконання різноманітних функцій написаних на різних мовах і приходять до висновків що їх функції, написані, наприклад, на PL/Python працюють в три-чотири рази швидше, ніж написані на PL/pgSQL. Про це, наприклад, [тут](https://stackoverflow.com/questions/4532229/any-difference-in-performance-compatibility-of-different-languages-in-postgresql), [тут](https://vk.com/@we_use_python-hranimye-procedury-na-python-v-postgresql) і [тут](https://habr.com/ru/company/postgrespro/blog/502254/).

Загальний висновок може бути таким:
- якщо Ваша функція переважно працює з БД - використовуйте SQL або PL/pgSQL - він інтегрований з БД, найнижчі витрати на запуск, серіалізацію і т. ін.
- якщо Вам необхідно в межах функції зробити щось інше (обробка рядків, регулярні вирази, використання спеціалізованих бібліотек і т .ін.) - використовуйте мову прграмування яка це робить краще.

Частіше Ви будете зустрічати функції, які написані на PL/pgSQL - тому ми поговоримо про нескладні приклади його використання. [Докладно про особливості цієї мови програмування можна познайомитись тут.](https://www.postgresql.org/docs/current/plpgsql.html)

##### Введення в PL/pgSQL

[Докладна документація тут.](https://www.postgresql.org/docs/15/plpgsql.html)

Синтаксис визначення функції - дуже схожий на те, що ми бачили раніше:

```SQL
CREATE FUNCTION func_name([arg_1,arg_2, ..., arg_n]) RETURNS data_type AS $$
BEGIN
-- LOGIC
END
$$ LANGUAGE plpgsql;
```


Для чого нам взагалі може знадобитись PL/pgSQL?
SQL - декларативна мова програмування, а PL/pgSQL надає нам процедурні можливості. 
Ми можемо виначати змінні, створювати цикли і інші розвинуті керуючі конструкції ('''IF ... THEN ... ELSIF ... THEN ... ELSE ... END IF''' - наприклад), створювати виключення - тобто використовувати повний набір інструментів звичних для будь-якої процедурної мови.

##### Функції які повертають скалярне значення.

Ми подивимось на прості функціх - для знайомства з синтаксисом.
Наприклад - функція яка повертае загальну кількість товарів на складі в штуках:

In [11]:
operation_query = """
CREATE OR REPLACE FUNCTION get_number_of_goods() RETURNS int AS $$
BEGIN
    RETURN SUM(units_in_stock)
    FROM products;
END
$$ LANGUAGE plpgsql
"""

final_set = execute_query(connection, operation_query)

Query executed successfully


Зверніть увагу, що в тілі функції ми використали замість SELECT -> RETURN.

Тепер перевіримо роботу:

In [12]:
operation_query = """
select get_number_of_goods()
"""

final_set = execute_read_query(connection, operation_query)
if final_set:
    for record in final_set:
        print(record)
else:
    print(final_set)

(3119,)


Давайте додамо вивод декількох параметрів. Пам'ятаєте - ми робили запит щоб дізнатись про диапазон дат роботи компанії який відображено в БД? Тобто найменшу і найбльшу дати виписування рахунків. Звертаю Вашу увагу, що кожен окремий оператор закінчується крапкою з комою.
Давайте зробимо для цього функцію:

In [14]:
operation_query = """
CREATE OR REPLACE FUNCTION get_start_stop_work_dates(OUT start_data DATE, OUT stop_data DATE) AS $$
BEGIN
    SELECT MIN(orders.order_date), MAX(orders.order_date)
    INTO start_data, stop_data
    FROM orders;
END
$$ LANGUAGE plpgsql
"""

final_set = execute_query(connection, operation_query)

Query executed successfully


Перевіримо:

In [15]:
operation_query = """
select get_start_stop_work_dates();
"""

final_set = execute_read_query(connection, operation_query)
if final_set:
    for record in final_set:
        print(record)
else:
    print(final_set)

('(1996-07-04,1998-05-06)',)


Як Ви бачите - ми отримуємо на самий "зручний" вид результату. Це тип RECORD.
Для того щоб отримати більш звичний результат - трохи перепишемо запит:

In [16]:
operation_query = """
select *
from get_start_stop_work_dates();
"""

final_set = execute_read_query(connection, operation_query)
if final_set:
    for record in final_set:
        print(record)
else:
    print(final_set)

(datetime.date(1996, 7, 4), datetime.date(1998, 5, 6))


В якості тренування давайте напишемо функцію яка має вхідні і вихідні параметри. Наприклад - отримує на вхід рядок і два інших рядки - субрядок який треба знайти в основному рядку в субрядок на який необхідно замінити всі знайдені входження.
[Опис синтаксису функції regexp_replace можна знайти тут.](https://www.postgresql.org/docs/15/functions-matching.html)
[Опис синтаксису функції concat() можна знайти тут](https://www.postgresql.org/docs/15/functions-string.html)
Ну і згадайте синтаксис регулярних виразів.

In [29]:
operation_query = """
CREATE OR REPLACE FUNCTION replace_substring(string CHARACTER, sub_str_old CHARACTER, sub_str_new CHARACTER, OUT result_string CHARACTER) AS $$
BEGIN
    result_string = regexp_replace(string, concat('(', sub_str_old, ')', '+'), sub_str_new, 'g');
END
$$ LANGUAGE plpgsql
"""

final_set = execute_query(connection, operation_query)

Query executed successfully


Пробуєм виконати:

In [30]:
operation_query = """
select *
from replace_substring('asdf-hhjg-trem---hhgfd-j', '-', '?');
"""

final_set = execute_read_query(connection, operation_query)
if final_set:
    for record in final_set:
        print(record)
else:
    print(final_set)

('asdf?hhjg?trem?hhgfd?j',)


Коли необхідно повернути декілька записів необхідно використовувати конструкцію RETURN QUERY.
Давайте напишемо функцію яка буде повертати всі товари в якійсь категрії товарів - в залежності від переданого в функцію значення category_id

In [32]:
operation_query = """
CREATE OR REPLACE FUNCTION get_all_products_of_category(category_id int) RETURNS SETOF products AS $$
BEGIN
    RETURN QUERY
    SELECT * 
    FROM products
    WHERE products.category_id = get_all_products_of_category.category_id;
END
$$ LANGUAGE plpgsql
"""

final_set = execute_query(connection, operation_query)

Query executed successfully


In [33]:
operation_query = """
select *
from get_all_products_of_category(2);
"""

final_set = execute_read_query(connection, operation_query)
if final_set:
    for record in final_set:
        print(record)
else:
    print(final_set)

(3, 'Aniseed Syrup', 1, 2, '12 - 550 ml bottles', 10.0, 13, 70, 25, 0)
(4, "Chef Anton's Cajun Seasoning", 2, 2, '48 - 6 oz jars', 22.0, 53, 0, 0, 0)
(5, "Chef Anton's Gumbo Mix", 2, 2, '36 boxes', 21.35, 0, 0, 0, 1)
(6, "Grandma's Boysenberry Spread", 3, 2, '12 - 8 oz jars', 25.0, 120, 0, 25, 0)
(8, 'Northwoods Cranberry Sauce', 3, 2, '12 - 12 oz jars', 40.0, 6, 0, 0, 0)
(15, 'Genen Shouyu', 6, 2, '24 - 250 ml bottles', 13.0, 39, 0, 5, 0)
(44, 'Gula Malacca', 20, 2, '20 - 2 kg bags', 19.45, 27, 0, 15, 0)
(61, "Sirop d'érable", 29, 2, '24 - 500 ml bottles', 28.5, 113, 0, 25, 0)
(63, 'Vegie-spread', 7, 2, '15 - 625 g jars', 43.9, 24, 0, 5, 0)
(65, 'Louisiana Fiery Hot Pepper Sauce', 2, 2, '32 - 8 oz bottles', 21.05, 76, 0, 0, 0)
(66, 'Louisiana Hot Spiced Okra', 2, 2, '24 - 8 oz jars', 17.0, 4, 100, 20, 0)
(77, 'Original Frankfurter grüne Soße', 12, 2, '12 boxes', 13.0, 32, 0, 15, 0)


##### Декларування змінних.

Для нас, як для людей вивчаючих python з його динамічною типізацією декларування змінних до їх використання може здаватися незручним і дещо громіздким. Але - це абсолютно звичайна дія для типізованих мов і, до речі, це дає змогу коду працювати швидше).
Як би там не було - PL/pgSQL вимагає декларувати змінні і їх тип до використання.
[Синтаксис докладно описаний тут.](https://www.postgresql.org/docs/15/plpgsql-declarations.html)


```SQL
CREATE FUNCTION func_name([arg_1,arg_2, ..., arg_n]) RETURNS data_type AS $$
DECLARE
    variable type_of_variable;
BEGIN
-- LOGIC
END
$$ LANGUAGE plpgsql;
```

Коли Вам необхідно в середині функції використовувати якісь змінні - їх треба описати, що і робиться в розділі DECLARE.
Ми з Вами раніше використовували деякі змінні в тілі функції, але ми їх описували в заявах вхідних і вихідних параметрів - тому PL/pgSQL вже мав інформацію про імена і типи.


##### Керуючі конструкції


Керуюча конструкция IF:

```SQL
IF expression THEN
    -- LOGIC
ELSIF expression THEN
    -- LOGIC
ELSIF expression THEN
    -- LOGIC
ELSE
    -- LOGIC
END IF;
```
Замість ELSIF можливо використовувати ELSEIF.

Для практики пропоную переписати логіку нашої раніше написаної функції для заміни субрядків в рядку. Давайте введемо умову - якшо довжина субрядка який необхідно шукати і заміняти (sub_str_old) по довжині більше ніж рядок в якому необхідно шукати входження - то ми в якості результату повертаємо sub_str_old.

In [43]:
operation_query = """
CREATE OR REPLACE FUNCTION replace_substring(string CHARACTER, sub_str_old CHARACTER, sub_str_new CHARACTER, OUT result_string CHARACTER) AS $$
BEGIN
    IF length(sub_str_old::text) > length(string::text) THEN
        result_string := sub_str_old;
    ELSE
        result_string := regexp_replace(string, concat('(', sub_str_old, ')', '{1}'), sub_str_new, 'g');
    END IF;
END
$$ LANGUAGE plpgsql
"""

final_set = execute_query(connection, operation_query)

Query executed successfully


Зверніть увагу на зміну у формуванні регулярного виразу і як це відображається на роботі функції.
Дивимось на результат

In [44]:
operation_query = """
select *
from replace_substring('+(38)-050-222-33-44', '-', '.');
"""

final_set = execute_read_query(connection, operation_query)
if final_set:
    for record in final_set:
        print(record)
else:
    print(final_set)

('+(38).050.222.33.44',)


Цикл WHILE

Тіло цикла виконується пока expression is True. Синтаксис:
```SQL
WHILE expression
LOOP
    -- LOGIC
END LOOP;
```
Ще одна форма (цикл закінчується коли expression is True):

```SQL
LOOP
    EXIT WHEN expression
    -- LOGIC
END LOOP;
```
Цикл FOR:

```SQL
FOR counter IN [REVERSE] start..stop [BY step]
LOOP
    -- LOGIC
END LOOP;
```
Все досить просто
- counter - змінна цикла, яка на кожному кроці приймає значення з диапазону start .. stop з необов'язковим параметром step
REVERSE - якщо є, то вказує що треба не збільшувати, а зменшувати counter на кожній ітерації.

Так як і в Python є оператор безумовного закінчення ітерації і переходу до наступної - ```CONTINUE WHEN expression```
Для того щоб перервати цикл і вийти за його межі у Python ми використовували оператор breack.
PL/pgSQL в цьому випадку використовує оператор EXIT.


Прлпоную написати функцію, яка буде повертати нам числа Фібоначі - класичне завдання для циклів)

In [45]:
operation_query = """
CREATE OR REPLACE FUNCTION fibo(n int) RETURNS int AS $$
DECLARE
	counter int := 0;
	i int = 0;
	j int = 1;
BEGIN
	IF n < i THEN
		RETURN 0;
	END IF;
	
	WHILE counter < n
	LOOP
		counter = counter + 1;
		select j, i+j into i, j;
	END LOOP;
	RETURN i;
END;
$$ LANGUAGE plpgsql
"""

final_set = execute_query(connection, operation_query)


Query executed successfully
